In this notebook, we prepare a dataset to be used to show how to perform text classification by fine-tuning a BERT-based model.

The data used here is obtained from the [Consumer Complaint Database](https://catalog.data.gov/dataset/consumer-complaint-database).

We download the entire dataset as a CSV file into the *data* local folder and then read it into a pandas dataframe.

In [1]:
import pandas as pd

df1 = pd.read_csv('./data/consumer_complaint_data.csv')

In [2]:
df1.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,08/09/2015,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,Company chooses not to provide a public response,Experian Information Solutions Inc.,NJ,08872,NaN,Consent not provided,Web,08/09/2015,Closed with non-monetary relief,Yes,No,1509954
1,01/29/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,"EQUIFAX, INC.",NY,10801,NaN,Consent not provided,Web,01/30/2019,Closed with non-monetary relief,Yes,NaN,3136759
2,08/19/2015,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,Company chooses not to provide a public response,WELLS FARGO & COMPANY,CA,94526,NaN,Consent not provided,Web,08/19/2015,Closed with explanation,Yes,No,1527601
3,03/04/2016,Credit card,NaN,Billing disputes,NaN,I am dissatisfied with the current outcome of ...,NaN,DISCOVER BANK,NV,891XX,NaN,Consent provided,Web,03/04/2016,Closed with explanation,Yes,Yes,1816726
4,03/18/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",PA,175XX,NaN,NaN,Referral,03/19/2013,Closed with explanation,Yes,No,358304


For our implementation, we use only the *Consumer complaint narrative* column, which we rename to *Complaint* and contains the textual information from the consumer complaints, and the *Product* column, which represents the financial products or services associated with a complaint.

In [3]:
df2 = df1[['Product', 'Consumer complaint narrative']]

In [4]:
df2.columns = ['Product', "Complaint"]

The dataset has approximately 1.4M rows, but a great portion of them has missing data in the *Complaint* column. Here we just drop all rows with missing data, and we end up with 472K rows.

In [5]:
df2.head()

,Product,Complaint
0,Credit reporting,NaN
1,"Credit reporting, credit repair services, or o...",NaN
2,Mortgage,NaN
3,Credit card,I am dissatisfied with the current outcome of ...
4,Mortgage,NaN


In [6]:
df2.shape

(1455769, 2)

In [7]:
df2.dropna(inplace=True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
df2.shape

(472991, 2)

There are 18 distinct values for the *Product* column, but some of them are very underrepresented. Also, there is a lot of overlapping among them.

We then consolidate the distinct values for the *Product* column into 6 distinct categories: *Credit Reporting*, *Debt Collection*, *Mortgage*, *Card Services*, *Loans*, and *Banking Services*.

In [9]:
df2['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    135833
Debt collection                                                                 103304
Mortgage                                                                         60068
Credit reporting                                                                 31588
Credit card or prepaid card                                                      30121
Student loan                                                                     24535
Credit card                                                                      18838
Checking or savings account                                                      18080
Bank account or service                                                          14885
Consumer Loan                                                                     9473
Vehicle loan or lease                                                             7786
Money transfer, virtual currency, or money 

In [10]:
df2.replace({'Product':
             {'Credit reporting, credit repair services, or other personal consumer reports': 'Credit Reporting',
              'Debt collection': 'Debt Collection',
              'Credit reporting': 'Credit Reporting',
              'Credit card': 'Card Services',
              'Bank account or service': 'Banking Services',
              'Credit card or prepaid card': 'Card Services',
              'Student loan': 'Loans',
              'Checking or savings account': 'Banking Services',
              'Consumer Loan': 'Loans',
              'Vehicle loan or lease': 'Loans',
              'Money transfer, virtual currency, or money service': 'Banking Services',
              'Payday loan, title loan, or personal loan': 'Loans',
              'Payday loan': 'Loans',
              'Money transfers': 'Banking Services',
              'Prepaid card': 'Card Services',
              'Other financial service': 'Other',
              'Virtual currency': 'Banking Services'}
            }, inplace= True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/generic.py:6702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex=regex,


In [11]:
df2 = df2[df2['Product'] != 'Other']

In [12]:
pd.DataFrame(df2['Product'].value_counts())

,Product
Credit Reporting,167421
Debt Collection,103304
Mortgage,60068
Card Services,50409
Loans,49573
Banking Services,41924


We need to represent data as numeric values for the model. Here we create a new column *Product_Label* that encodes the information from the *Product* column into numeric values.

We need to do something similar for the textual information from the *Complaint* column, but as this is dependent of the model architecture, this is done in the subsequent notebook.

In [13]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df2['Product_Label'] = enc.fit_transform(df2['Product'])

In [14]:
df2.head()

,Product,Complaint,Product_Label
3,Card Services,I am dissatisfied with the current outcome of ...,1
7,Credit Reporting,On XX/XX/18 an item showed up on my credit rep...,2
8,Debt Collection,Saw on my credit report that there is a collec...,3
9,Credit Reporting,Capital one auto finacing XX/XX/2018 unknown i...,2
11,Card Services,I own a small trucking company. One of my truc...,1


In [15]:
df2.iloc[4]['Complaint']

'I own a small trucking company. One of my trucks needed a repair. I hired a mechanic to do the repair on XX/XX/XXXX. He told us he installed new batteries and a new starter on the truck. The mechanic stated his work was guaranteed and I paid him with my TD Bank XXXX credit card {$2300.00} for this repair. When my driver went to start the truck for work the next day, the truck did not work/start again. We tried calling/texting mechanic to let him know the truck was not working even though the previous day he said he fixed the truck. I have the texts and phone records showing we were reaching out to the mechanic with no response from him. I had to hire another mechanic to fix truck again and found out the parts put in my truck by the first mechanic were wrong. I filed a dispute on XX/XX/XXXX for the above transaction with my TD Bank XXXX credit card. Even though I paid the first mechanic {$2300.00}, my dispute amount was for only {$1300.00} because that amount is for the wrong parts. Th

We can further preprocess the data, by trying to decrease the vocabulary size for the text. Here we perform a light text preprocessing, by removing punctuation, removing the masked information (*XXX…* patterns), removing extra spaces and finally normalize everything to lowercase.

In [16]:
import string

table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
df2['Complaint'] = df2['Complaint'].str.translate(table)
df2['Complaint'] = df2['Complaint'].str.replace('X+', '')
df2['Complaint'] = df2['Complaint'].str.replace(' +', ' ')
df2['Complaint'] = df2['Complaint'].str.lower()
df2['Complaint'] = df2['Complaint'].str.strip()

In [17]:
df2.iloc[4]['Complaint']

'i own a small trucking company one of my trucks needed a repair i hired a mechanic to do the repair on he told us he installed new batteries and a new starter on the truck the mechanic stated his work was guaranteed and i paid him with my td bank credit card 2300 00 for this repair when my driver went to start the truck for work the next day the truck did not work start again we tried calling texting mechanic to let him know the truck was not working even though the previous day he said he fixed the truck i have the texts and phone records showing we were reaching out to the mechanic with no response from him i had to hire another mechanic to fix truck again and found out the parts put in my truck by the first mechanic were wrong i filed a dispute on for the above transaction with my td bank credit card even though i paid the first mechanic 2300 00 my dispute amount was for only 1300 00 because that amount is for the wrong parts the credit card company requested proof of records and a

There is some text in the *Complaint* column that has 0 or very few words, which represents about 1,000 rows in the dataset. Here we consider the minimum of 5 words for the text to have some useful information.

In [18]:
lengths = [len(df2.iloc[i]['Complaint'].split()) for i in range(len(df2))]
print(max(lengths))
print(min(lengths))

5958
0


In [19]:
df2 = df2[[l >= 5 for l in lengths]]

In [20]:
df2.shape

(471936, 3)

In [21]:
pd.DataFrame(df2['Product'].value_counts())

,Product
Credit Reporting,166899
Debt Collection,103144
Mortgage,60050
Card Services,50385
Loans,49548
Banking Services,41910


We then save the preprocessed dataset, and another one corresponding to a 10% sample.

In [22]:
df2.to_csv('./data/consumer_complaint_data_prepared.csv', index=False)

In [23]:
df2.sample(n=int(len(df2)*0.1), random_state=111).to_csv('./data/consumer_complaint_data_sample_prepared.csv', index=False)